In [10]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import  matplotlib.pyplot as plt
from  sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from imblearn.over_sampling import SMOTE,ADASYN
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import  make_pipeline,Pipeline
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import f_regression
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.ensemble import GradientBoostingRegressor





In [11]:
df=pd.read_csv(r"C:\Users\hp\Desktop\house_sale.csv")

In [12]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
def data_preprocessor(X,y):
        """Function to Prepocess the data """
        numeric_transformer = Pipeline(
                steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
            )

        categorical_transformer = OneHotEncoder(handle_unknown="ignore")
        preprocessor = ColumnTransformer(
                transformers=[
                    ("numeric", numeric_transformer, X.select_dtypes(np.number).columns.tolist()),
                    ("category", categorical_transformer,X.select_dtypes("object").columns.tolist()),
                ],remainder="passthrough"
            )
        return preprocessor
def result_evaluator_regressor(model,xtest,ytest):
         ypred_model=model.predict(xtest)
         r2score=r2_score(ytest,ypred_model)
         mse=mean_absolute_error(ytest,ypred_model)
         return r2score,mse

In [13]:
import random
print(random.randint(1000,9999))

5491


In [14]:
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
X=df.drop(columns=["SalePrice"])
y=df["SalePrice"]

In [15]:
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score

In [16]:
def result_evaluator_classfication(model,xtest,ytest):
         ypred=model.predict(xtest)
         acuracy=accuracy_score(ytest,ypred)
         clrep=classification_report(ytest,ypred)
         return clrep,acuracy

In [17]:
df.head()

,Unnamed: 0,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,60,RL,65.0,8450,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,2,2008,WD,Normal,208500
1,1,20,RL,80.0,9600,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,5,2007,WD,Normal,181500
2,2,60,RL,68.0,11250,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,9,2008,WD,Normal,223500
3,3,70,RL,60.0,9550,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,2,2006,WD,Abnorml,140000
4,4,60,RL,84.0,14260,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,12,2008,WD,Normal,250000


In [18]:
fp=RandomForestRegressor()

In [19]:
def svm_Regressor(X,y,start,end,parms,f):

        Listing=[]

        preprocessor=data_preprocessor(X,y)
        # st.write("In main FUnction ")
        xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=45)
    #     parms={
    #     "classifier__splitter":["random", "best"],
    #     "classifier__max_features":["sqrt", "log2", None],
    #     'classifier__max_depth':[10,15,20,25,30],
    # }   
        for i in range(start,end):
        # ****************Feature Seclortot**********************************************
                feature_selector = Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SelectKBest(f_regression,k=i))])
                feature_selector.fit(xtrain,ytrain)
        # ****************Model Seclortot**********************************************
                model_selector = Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SelectKBest(f_regression,k=i)),
                    ("classifier",  f)]
                )
                model_selector.fit(xtrain,ytrain)
        # *********************************Hyper Parametet***********************************
                grid=GridSearchCV(model_selector,parms,cv=4,n_jobs=-1,verbose=3)
                grid.fit(xtrain,ytrain)
                feature=grid.best_params_
                model=grid.best_estimator_

                
                result_parameter,mae_parameter=result_evaluator_regressor(model,xtest,ytest)
               
                print(f"R2 score with best parameter{result_parameter}")
                print(f"Mean Absoulte Error with best parameter{mae_parameter}")
           
  
        #****************************Result Generation ******************************
                result,mae=result_evaluator_regressor(model_selector,xtest,ytest)
                

                print(f"Accuracy without  Best parameter{result}")
                print(f"Mean Absoulte Error without  best parameter{mae}")
            
        #*********************************Working on features****************************
                xopt=feature_selector.get_feature_names_out()
                feature_selection=[]
                for x in xopt:
                    feature_selection.append(x.split("__")[1])
                # st.write(feature_selection)
                print("The feature Selection are as follow-:")
                print(feature_selection)
                print(feature)
                print(feature_selection)
                Listing.append({
                    "i":i,
                    "result":result,
                    "mae":mae,
                    "result_parameter":result_parameter,
                    "mae_parameter":mae_parameter,
                #     "Error_model":result_model,
                    "columns":feature_selection,
                    "parameter":feature
                })
        print(Listing)

        return Listing,model

In [20]:
parms={}
listing,model=svm_Regressor(X,y,1,10,parms,fp)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
R2 score with best parameter0.682695870160644
Mean Absoulte Error with best parameter31880.90748015832
Accuracy without  Best parameter0.6824738568771969
Mean Absoulte Error without  best parameter31895.21456134558
The feature Selection are as follow-:
['OverallQual']
{}
['OverallQual']
Fitting 4 folds for each of 1 candidates, totalling 4 fits
R2 score with best parameter0.7445337070471845
Mean Absoulte Error with best parameter28915.10070695714
Accuracy without  Best parameter0.7458404333722549
Mean Absoulte Error without  best parameter28818.78513447026
The feature Selection are as follow-:
['OverallQual', 'GrLivArea']
{}
['OverallQual', 'GrLivArea']
Fitting 4 folds for each of 1 candidates, totalling 4 fits
R2 score with best parameter0.7975501113118314
Mean Absoulte Error with best parameter26469.05454697291
Accuracy without  Best parameter0.8020923198572782
Mean Absoulte Error without  best parameter26403.476391167143
The

In [21]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Unnamed: 0', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'Tot...
                                                   'HouseStyle', 'RoofStyle',
                                                   'RoofMatl', 'Exterior1st',
                                                   'Exterior2nd', 'MasVnrType',
                                                   'ExterQual', 'ExterCond',
                                                   'Foundation', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', ...])])),
                ('feature',
                 SelectKBest(k=9,
                             score_func=<function f_regression at 0x000001DC6F12FC10>)),
                ('classifier', RandomForestRegressor())])

In [30]:
result=pd.DataFrame(listing)
result


,i,result,mae,result_parameter,mae_parameter,columns,parameter
0,1,0.682474,31895.214561,0.682696,31880.907480,[OverallQual],{}
1,2,0.745840,28818.785134,0.744534,28915.100707,"[OverallQual, GrLivArea]",{}
2,3,0.802092,26403.476391,0.797550,26469.054547,"[OverallQual, GrLivArea, GarageCars]",{}
3,4,0.816532,23719.382827,0.818839,23852.067903,"[OverallQual, GrLivArea, GarageCars, GarageArea]",{}
4,5,0.841122,22162.088466,0.838488,22368.698016,"[OverallQual, TotalBsmtSF, GrLivArea, GarageCa...",{}
5,6,0.845382,22048.917099,0.842604,21968.408703,"[OverallQual, TotalBsmtSF, GrLivArea, GarageCa...",{}
6,7,0.840378,22310.462330,0.844348,21889.486912,"[OverallQual, TotalBsmtSF, 1stFlrSF, GrLivArea...",{}
7,8,0.845860,21932.825882,0.844563,22010.577330,"[OverallQual, TotalBsmtSF, 1stFlrSF, GrLivArea...",{}
8,9,0.850529,21719.499115,0.850474,21676.208900,"[OverallQual, TotalBsmtSF, 1stFlrSF, GrLivArea...",{}


In [32]:
z=result.sort_values(by=["i","result_parameter",],ascending=[False,True])
z

,i,result,mae,result_parameter,mae_parameter,columns,parameter
8,9,0.850529,21719.499115,0.850474,21676.208900,"[OverallQual, TotalBsmtSF, 1stFlrSF, GrLivArea...",{}
7,8,0.845860,21932.825882,0.844563,22010.577330,"[OverallQual, TotalBsmtSF, 1stFlrSF, GrLivArea...",{}
6,7,0.840378,22310.462330,0.844348,21889.486912,"[OverallQual, TotalBsmtSF, 1stFlrSF, GrLivArea...",{}
5,6,0.845382,22048.917099,0.842604,21968.408703,"[OverallQual, TotalBsmtSF, GrLivArea, GarageCa...",{}
4,5,0.841122,22162.088466,0.838488,22368.698016,"[OverallQual, TotalBsmtSF, GrLivArea, GarageCa...",{}
3,4,0.816532,23719.382827,0.818839,23852.067903,"[OverallQual, GrLivArea, GarageCars, GarageArea]",{}
2,3,0.802092,26403.476391,0.797550,26469.054547,"[OverallQual, GrLivArea, GarageCars]",{}
1,2,0.745840,28818.785134,0.744534,28915.100707,"[OverallQual, GrLivArea]",{}
0,1,0.682474,31895.214561,0.682696,31880.907480,[OverallQual],{}


In [22]:
from imblearn.over_sampling import SMOTE,ADASYN,RandomOverSampler
from imblearn.under_sampling import NearMiss,RandomUnderSampler

In [23]:
smt=SMOTE(random_state=23)
oversampler = RandomOverSampler()

In [24]:
a=0.6467
round(a,2)*100


65.0

In [25]:
def svm_Regressor(X,y,start,end,parms,sampling="SMOTE"):

        preprocessor=data_preprocessor(X,y)
        Listing=[]
        # st.write("In main FUnction ")
        xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=45)
    #     parms={
    #     "classifier__splitter":["random", "best"],
    #     "classifier__max_features":["sqrt", "log2", None],
    #     'classifier__max_depth':[10,15,20,25,30],
    # }
        for i in range(start,end):
        # ****************Feature Seclortot**********************************************
                feature_selector = Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SelectKBest(f_regression,k=i))])
                feature_selector.fit(xtrain,ytrain)
        # ****************Model Seclortot**********************************************
                model_selector = Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SelectKBest(f_regression,k=i)),
                    ("oversampler" ,RandomOverSampler()),
                    ("classifier",  RandomForestRegressor())]
                )
                model_selector.fit(xtrain,ytrain)
        # *********************************Hyper Parametet***********************************
                grid=GridSearchCV(model_selector,parms,cv=4,n_jobs=-1,verbose=3)
                grid.fit(xtrain,ytrain)
                feature=grid.best_params_
                model=grid.best_estimator_

                
                result_parameter,mae_parameter=result_evaluator_regressor(model,xtest,ytest)
               
                print(f"R2 score with best parameter{result_parameter}")
                print(f"Mean Absoulte Error with best parameter{mae_parameter}")
           
  
        #****************************Result Generation ******************************
                result,mae=result_evaluator_regressor(model_selector,xtest,ytest)
                

                print(f"Accuracy without  Best parameter{result}")
                print(f"Mean Absoulte Error without  best parameter{mae}")
            
        #*********************************Working on features****************************
                xopt=feature_selector.get_feature_names_out()
                feature_selection=[]
                for x in xopt:
                    feature_selection.append(x.split("__")[1])
                # st.write(feature_selection)
                # print("The feature Selection are as follow-:")
                # print(feature_selection)
                # print(feature)
                # print(feature_selection)
                Listing.append({
                    "i":i,
                    "result":result,
                    "mae":mae,
                    "result_parameter":result_parameter,
                    "mae_parameter":mae_parameter,
                #     "Error_model":result_model,
                    "columns":feature_selection,
                    "parameter":feature,
                    "sampling":sampling,
                })
        print(Listing)

        return Listing,model
parms={}
listing,model=svm_Regressor(X,y,1,10,parms)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomOverSampler()' (type <class 'imblearn.over_sampling._random_over_sampler.RandomOverSampler'>) doesn't

In [ ]:
preprocessor=data_preprocessor(X,y)
sm=Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("feature", SMOTE())
          ])
X,y=sm.fit(X,y)

In [ ]:
X

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Unnamed: 0', 'MSSubClass', 'LotFrontage',
                                  'LotArea', 'OverallQual', 'OverallCond',
                                  'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
                                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                                  'LowQualFinSF', 'GrLivAre...
                                 ['MSZoning', 'Street', 'Alley', 'LotShape',
                                  'LandContour', 'Utilities', 'LotConfig',
                                  'LandSlope', 'Neighborhood', 'Condition1',
                                  'Condition2', 'BldgType', 'HouseStyle',
                                  'RoofStyle', 'RoofMatl', 'Exterior1st',
                                  'Exterior2nd', 'MasVnrType', 'ExterQual',
                                  'ExterCond', 'Foundation', 'BsmtQual',
                                  'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Heating', 'HeatingQC',
                                  'CentralAir', 'Electrical', ...])])

In [ ]:
def svm_Regressor(X,y,start,end,parms,sampling="SMOTE"):
        Listing=[]

        preprocessor=data_preprocessor(X,y)
        Listing=[]

        if sampling=="SMOTE":
               sm=Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SMOTE())
                    ])
               X,y=sm.fit(X,y)
        # elif sampling=="ADASYN":
        #         X,y=ADASYN().fit_resample(X,y)
        # elif sampling=="RandomOverSampler":
        #         X,y=RandomOverSampler().fit_resample(X,y)
        # elif sampling=="RandomUnderSampler":
        #         X,y=RandomUnderSampler().fit_resample(X,y)
        # elif sampling=="NearMiss":
        #         X,y=NearMiss().fit_resample(X,y)
        # print(X.shape,y.shape)
        
        # st.write("In main FUnction ")
        xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=45)
    #     
        for i in range(start,end):
        # ****************Feature Seclortot**********************************************
                feature_selector = Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SelectKBest(f_regression,k=i))])
                feature_selector.fit(xtrain,ytrain)
        # ****************Model Seclortot**********************************************
                model_selector = Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SelectKBest(f_regression,k=i)),
                    ("classifier",  RandomForestRegressor())]
                )
                model_selector.fit(xtrain,ytrain)
        # *********************************Hyper Parametet***********************************
                grid=GridSearchCV(model_selector,parms,cv=4,n_jobs=-1,verbose=3)
                grid.fit(xtrain,ytrain)
                feature=grid.best_params_
                model=grid.best_estimator_
                result_parameter,mae_parameter=result_evaluator_regressor(model,xtest,ytest)
                print(f"R2 score with best parameter{result_parameter}")
                print(f"Mean Absoulte Error with best parameter{mae_parameter}")
           
  
        #****************************Result Generation ******************************
                result,mae=result_evaluator_regressor(model_selector,xtest,ytest)
                

                print(f"Accuracy without  Best parameter{result}")
                print(f"Mean Absoulte Error without  best parameter{mae}")
            
        #*********************************Working on features****************************
                xopt=feature_selector.get_feature_names_out()
                feature_selection=[]
                for x in xopt:
                    feature_selection.append(x.split("__")[1])
                Listing.append({
                    "i":i,
                    "result":result,
                    "mae":mae,
                    "result_parameter":result_parameter,
                    "mae_parameter":mae_parameter,
                #     "Error_model":result_model,
                    "columns":feature_selection,
                    "parameter":feature,
                    "sampling":sampling,
                })
        print(Listing)

        return Listing,model
parms={}
listing,model=svm_Regressor(X,y,1,10,parms)

AttributeError: 'ColumnTransformer' object has no attribute 'select_dtypes'

In [ ]:
print(pd.DataFrame(listing).to_csv("listing5.csv"))

None


In [ ]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Unnamed: 0', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2n...
                                                   'HouseStyle', 'RoofStyle',
                                                   'RoofMatl', 'Exterior1st',
                                                   'Exterior2nd', 'MasVnrType',
                                                   'ExterQual', 'ExterCond',
                                                   'Foundation', 'BsmtQual',
                                                   'BsmtCond', 'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', ...])])),
                ('feature',
                 SelectKBest(k=9,
                             score_func=<function f_regression at 0x00000224AEAAA040>)),
                ('classifier', RandomForestRegressor())])

POlynomail regression

In [ ]:
length=pd.DataFrame(listing)["columns"][0][2:5]
# print(len(length))
# print(length)

In [ ]:
def result_evaluator_classfication(model,xtest,ytest):
         ypred=model.predict(xtest)
         acuracy=accuracy_score(ytest,ypred)
         clrep=classification_report(ytest,ypred)
         return clrep,acuracy

In [ ]:
from sklearn.svm import SVC

In [3]:
import datetime

today_date = datetime.date.today()
print("Today's date is:", today_date)


Today's date is: 2023-03-31


In [ ]:
def svc_Regressor(X,y,start,end,parms):

        Listing=[]

        preprocessor=data_preprocessor(X,y)
        # st.write("In main FUnction ")
        xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=45)
    #     parms={
    #     "classifier__splitter":["random", "best"],
    #     "classifier__max_features":["sqrt", "log2", None],
    #     'classifier__max_depth':[10,15,20,25,30],
    # }   
        for i in range(start,end):
        # ****************Feature Seclortot**********************************************
                feature_selector = Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SelectKBest(f_regression,k=i))])
                feature_selector.fit(xtrain,ytrain)
        # ****************Model Seclortot**********************************************
                model_selector = Pipeline(
                    steps=[("preprocessor", preprocessor),
                    ("feature", SelectKBest(f_regression,k=i)),
                    ("classifier",  SVC())]
                )
                model_selector.fit(xtrain,ytrain)
        # *********************************Hyper Parametet***********************************
                grid=GridSearchCV(model_selector,parms,cv=4,n_jobs=-1,verbose=3)
                grid.fit(xtrain,ytrain)
                feature=grid.best_params_
                model=grid.best_estimator_

                # st.markdown("_"*200)
                result_parameter,mae_parameter=result_evaluator_regressor(model,xtest,ytest)
                # ypred_model=model.predict(xtest)
                # result_model=r2_score(ytest,ypred_model)
                # st.markdown("_"*200)
                print(f"R2 score with best parameter{result_parameter}")
                print(f"Mean Absoulte Error with best parameter{mae_parameter}")
                # ypred_model=model.predict(xtest)
                # result_model=r2_score(ytest,ypred_model)
                # st.write(f"Accuracy with best parameter{result_parameter}")
                # print(f"R2score of Result Modelwith best estimator is : {result_model}")
        #****************************Result Generation ******************************
                result,mae=result_evaluator_regressor(model_selector,xtest,ytest)
                # ypred=model_selector.predict(xtest)
                # result=r2_score(ytest,ypred)

                print(f"Accuracy without  Best parameter{result}")
                print(f"Mean Absoulte Error without  best parameter{mae}")
                # print(f"R2score is without estimator {result}")
                # print(confusion_matrix(ypred,ytest))
                # print(classification_report(ypred,ytest))
        #*********************************Working on features****************************
                xopt=feature_selector.get_feature_names_out()
                feature_selection=[]
                for x in xopt:
                    feature_selection.append(x.split("__")[1])
                # st.write(feature_selection)
                print("The feature Selection are as follow-:")
                print(feature_selection)
                print(feature)
                print(feature_selection)
                Listing.append({
                    "i":i,
                    "result":result,
                    "mae":mae,
                    "result_parameter":result_parameter,
                    "mae_parameter":mae_parameter,
                #     "Error_model":result_model,
                    "columns":feature_selection,
                    "parameter":feature
                })
        print(Listing)

        return Listing,model

In [ ]:
df=pd.read_csv(r"C:\Users\hp\Desktop\file1.csv")
X=df.drop(columns="Class")
# y=df["h1n1_vaccine"]
le=LabelEncoder()
y=le.fit_transform(df["Class"])
le=LabelEncoder()
y=le.fit_transform(df["Class"])

In [ ]:
parms={}
listing,model=svc_Regressor(X,y,1,3,parms)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
R2 score with best parameter-0.2820512820512824
Mean Absoulte Error with best parameter0.22
Accuracy without  Best parameter-0.2820512820512824
Mean Absoulte Error without  best parameter0.22
The feature Selection are as follow-:
['Recency']
{}
['Recency']
Fitting 4 folds for each of 1 candidates, totalling 4 fits
R2 score with best parameter-0.3209013209013212
Mean Absoulte Error with best parameter0.22666666666666666
Accuracy without  Best parameter-0.3209013209013212
Mean Absoulte Error without  best parameter0.22666666666666666
The feature Selection are as follow-:
['Recency', 'Monetary']
{}
['Recency', 'Monetary']
[{'i': 1, 'result': -0.2820512820512824, 'mae': 0.22, 'result_parameter': -0.2820512820512824, 'mae_parameter': 0.22, 'columns': ['Recency'], 'parameter': {}}, {'i': 2, 'result': -0.3209013209013212, 'mae': 0.22666666666666666, 'result_parameter': -0.3209013209013212, 'mae_parameter': 0.22666666666666666, 'column